# This notebook explains how to obtain high-quality price data at daily frequency from US listed stocks 

In [ ]:
from IPython.display import Image

## Getting data: WRDS, CRSP and tickers for the S&P 500 universe.

For this project, we make use of `CRSP` data. 

The acronym `CRSP` stands for `Center for Research in Security Prices` and you can find more information about them here: https://www.crsp.org/
They are a respected data provider that are behind a paywall. Unfortunately, high-quality financial data is known to be unaffordably expensive, but luckily Imperial College has a license. Precisely, you can access `CRSP` data via `WRDS`, being short for `Wharton Research Data Services`. WRDS understand themselves to be "the global standard for business research". We will not question the details of this statement but they certainly do offer data that you also can make use for you university projects and disseration all the way up to PhD and Postdoc research. You can find more information about them here: https://wrds-www.wharton.upenn.edu. 

In order to access `CRSP` data via `WRDS`, you need to follow these steps: 

1) Apply for a `WRDS` account via Imerial College's library services here:
https://www.imperial.ac.uk/admin-services/library/subject-support/business/wrds/

2) Log on your `WRDS` account:
https://wrds-www.wharton.upenn.edu/login/

3) Upon logging in, you will see a menu that looks like the following. Click on `CRSP`.

In [ ]:
Image(filename="./../figures/WRDS_overview.png")

4) Under `Quarterly Update`, click on `Stock - Version 2`.

In [ ]:
Image(filename="./../figures/CRSP1_select_quarterly.png")

5) Under `Stock - Version 2`, click on `Daily Stock File`.

In [ ]:
Image(filename="/home/bjs/Desktop/vcs/ReCoDE-DecodingMarketSignals/figures/CRSP2.png")

5) Finally, you can see a mask that allows you to select a date range of your choosing as well as a mask to select the identifyer of the stock you wish to query data for. I opten for `Ticker`. You will find a list of tickers for the S&P 500 components as of the end of 2023 in the text file `data/SP500_tickers_one_per_line.txt` in the file directory of this exemplar. If you work for a bank or financial institution, they will ususlly have an `Operations` team that tracks any tickers at any exchange they trade on. Tickers, as well as index components can change in time, as members will be included and others excluded from an index. Hence, the composition of the ticker list can change. I obtained the tickers from https://www.slickcharts.com/sp500 on 4 December 2023, which is a publically available source. I encourage you to scrape the data from the web, comparing them to the tickers contained and updating any changes.

Now back to WRDS. If you want to query data for more than one stock, you do not need to enter them manually. Instead, you can upload a file containing the identifiers. `WRDS` demands one identifier, here ticker, per line of the file you upload. You can then go ahead and select the data you wish to query for each of the tickers. For the purpose of this exempolar, we need to obatin "open price", "high price", "low price" and "close price", and go all the way back from `1980` to the most recent date available, here, `31 December 2023`.
You can choose to compress the output file which is suggested for larger data requests. 

Clicking on `Submit Form` will query the data. You will receive an email notifying you once the process is complete and can then progress importing it to the program of choice to conduct your research on the data. 

In [ ]:
Image(
    filename="/home/bjs/Desktop/vcs/ReCoDE-DecodingMarketSignals/figures/CRSP3_select_data.png"
)

# END